In [1]:
# api 설치
%pip install -r ./requirements.txt

  Using cached fastapi-0.116.2-py3-none-any.whl.metadata (28 kB)
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.24.5-py3-none-any.whl.metadata (13 kB)
  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached kiwipiepy-0.21.0-cp312-cp312-win_amd64.whl.metadata (1.3 kB)
  Using cached wordcloud-1.9.4-cp312-cp312-win_amd64.whl.metadata (3.5 kB)
  Using cached tokenizers-0.19.1-cp312-none-win_amd64.whl.metadata (6.9 kB)
  Using cached aiohttp-3.12.15-cp312-cp312-win_amd64.whl.metadata (7.9 kB)
Using cached fastapi-0.116.2-py3-none-any.whl (95 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
Using cached huggingface_hub-0.24.5-py3-none-any.whl (417 kB)
Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
Using cached kiwipiepy-0.21.0-cp312-cp312-win_amd64.whl (2.4 MB)
Using cached wordcloud-1.9.4-cp312-cp312-win_amd64.whl (301 kB)
Using cached tokenizers-0.19.1-cp312-none-win_amd64.whl (2.2 MB)
Usi

In [2]:
from fastapi import FastAPI
from pydantic import BaseModel
from contextlib import asynccontextmanager
from module.model import load_models, predict
from module.chatbot import get_chatbot_chain
from module.wordcount import count_tokens
import logging

c:\Users\kase\Desktop\todak_hackerton\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'langchain_openai'

In [ ]:
# 로거 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("my_app")

In [12]:
# lifespan으로 모델 로드 관리
@asynccontextmanager
async def lifespan(app: FastAPI):
    global models, tokenizers, device
    models, tokenizers, device = load_models()
    yield
    models.clear()
    tokenizers.clear()

In [ ]:
app = FastAPI(lifespan=lifespan)

In [ ]:

class ChatRequest(BaseModel):
    message: str

class ChatResponse(BaseModel):
    reply: str

@app.post("/api/v1/chatbot", response_model=ChatResponse)
def chat(req: ChatRequest):
    logger.info(f"호출내용 : {req.message}")
    chatbot = get_chatbot_chain()
    response = chatbot.run(message=req.message)
    return ChatResponse(reply=response)

- 응답예시{json 형식}  
{
  "reply": "오늘 하루 많이 힘드셨겠어요."
}

In [ ]:
class TextInput(BaseModel):
    text: str

@app.get("/api/v1/channels/{channelId}/reports/frequency")
def analyze_text(channelId: str, data: TextInput):
    # 1) 단어 카운트
    counter = count_tokens(data.text)
    token_freqs = [{"token": tok, "count": cnt} for tok, cnt in counter.most_common()]

    # 2) 감지 모델 예측
    depression_score = predict(data.text, models["depression"], tokenizers["depression"], device)
    anxiety_score = predict(data.text, models["anxiety"], tokenizers["anxiety"], device)

    # 3) 통합 Response
    return {
        "channelId": channelId,
        "depression_score": str(depression_score),
        "anxiety_score": str(anxiety_score),
        "report": {tok["token"]: tok["count"] for tok in token_freqs}
    }


- 응답예시(json형식)  
""" {
  "channelId": "123",  
  "input": "오늘은 기분이 우울하고 불안하다.",  
  "tokens": [  
    {"token": "기분", "count": 1},  
    {"token": "우울하고", "count": 1},  
    {"token": "불안하다", "count": 1}  
  ],  
  "predictions": {  
    "depression": 2,  
    "anxiety": 1  
  }  
} """

In [ ]:
# 직동확인 부분(최종 확인후 주석처리 혹은 삭제할 것)
@app.get("/")
def root():
    return {"message": "KLUEBERT API is running", "models": ["depression", "anxiety"]}